In [1]:
from django.template.base import Template
from django.template.context import Context
from cmj.utils import Manutencao
from cmj.loa import models
from io import StringIO
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import fitz
import csv
import re
m = Manutencao()
#m.desativa_auto_now()
m.desativa_signals()

In [3]:
for sr in ScrapRecord.objects.filter(ano=2024).exclude(codigo=''
    ).values('ano', 'orgao', 'mes'
    ).order_by('-ano', 'orgao', '-mes').annotate(count=Count('ano')):
    print(sr)

{'ano': 2024, 'orgao': '01', 'mes': 9, 'count': 68}
{'ano': 2024, 'orgao': '01', 'mes': 8, 'count': 158}
{'ano': 2024, 'orgao': '01', 'mes': 7, 'count': 70}
{'ano': 2024, 'orgao': '01', 'mes': 6, 'count': 254}
{'ano': 2024, 'orgao': '01', 'mes': 5, 'count': 220}
{'ano': 2024, 'orgao': '01', 'mes': 4, 'count': 206}
{'ano': 2024, 'orgao': '01', 'mes': 3, 'count': 160}
{'ano': 2024, 'orgao': '01', 'mes': 2, 'count': 152}
{'ano': 2024, 'orgao': '01', 'mes': 1, 'count': 82}
{'ano': 2024, 'orgao': '03', 'mes': 9, 'count': 816}
{'ano': 2024, 'orgao': '03', 'mes': 8, 'count': 1610}
{'ano': 2024, 'orgao': '03', 'mes': 7, 'count': 1694}
{'ano': 2024, 'orgao': '03', 'mes': 6, 'count': 1786}
{'ano': 2024, 'orgao': '03', 'mes': 5, 'count': 1924}
{'ano': 2024, 'orgao': '03', 'mes': 4, 'count': 1780}
{'ano': 2024, 'orgao': '03', 'mes': 3, 'count': 1702}
{'ano': 2024, 'orgao': '03', 'mes': 2, 'count': 1372}
{'ano': 2024, 'orgao': '03', 'mes': 1, 'count': 1444}
{'ano': 2024, 'orgao': '04', 'mes': 9, 'c

In [4]:
scraps = {l.ano: {} for l in Loa.objects.all()}
for sr in ScrapRecord.objects.filter(ano__in=scraps.keys()).exclude(codigo=''):
    sra = scraps[sr.ano]
    
    if sr.codigo not in sra:
        sra[sr.codigo] = {
            'csv': None,
            'html': None
        }
    
    sra[sr.codigo][sr.metadata['url_dict']['format']] = sr

In [5]:
scraps.keys()

dict_keys([2025, 2024, 2023, 2022, 2021, 2020, 2019])

In [6]:
len(scraps[2024].keys())

20111

In [7]:
item = list(scraps[2024].values())[0]

In [8]:
item

{'csv': <ScrapRecord: ScrapRecord object (150)>,
 'html': <ScrapRecord: ScrapRecord object (151)>}

In [9]:
for ano, scrap_anual in scraps.items():
    for codigo, item in scrap_anual.items():
        break

In [10]:
codigo

'422863'

In [11]:
item

{'csv': <ScrapRecord: ScrapRecord object (1762)>,
 'html': <ScrapRecord: ScrapRecord object (1763)>}

In [12]:
item['csv'].__dict__

{'_state': <django.db.models.base.ModelState at 0x72dec7a37820>,
 'id': 1762,
 'metadata': {'url_dict': {'url': 'http://prefeituradejatai.sigepnet.com.br/transparencia/exportacao/depesa_paga_detalhes.php?extensao=CSV&codigo={codigo}',
   'name': 'despesa_paga_detalhes_exportacao',
   'type': 'detail',
   'format': 'csv',
   'params': ['codigo']},
  'item_list': ['422863',
   '13/12/2019',
   'ACACIO MICENA COUTINHO',
   '310.***.***-87',
   'Despesas a Pagar',
   '6.153,84']},
 'mes': 12,
 'ano': 2019,
 'orgao': '17',
 'codigo': '422863',
 'url': 'http://prefeituradejatai.sigepnet.com.br/transparencia/exportacao/depesa_paga_detalhes.php?extensao=CSV&codigo=422863',
 'content': <memory at 0x72dec9259780>,
 'modified': datetime.datetime(2024, 9, 15, 6, 8, 59, 323038, tzinfo=<UTC>),
 'parent_id': 1761}

In [13]:
item['html'].__dict__

{'_state': <django.db.models.base.ModelState at 0x72dec7a378b0>,
 'id': 1763,
 'metadata': {'url_dict': {'url': 'http://prefeituradejatai.sigepnet.com.br/transparencia/despesa_paga_detalhes.php?codigo={codigo}',
   'name': 'despesa_paga_detalhes',
   'type': 'detail',
   'format': 'html',
   'params': ['codigo']},
  'item_list': ['422863',
   '13/12/2019',
   'ACACIO MICENA COUTINHO',
   '310.***.***-87',
   'Despesas a Pagar',
   '6.153,84']},
 'mes': 12,
 'ano': 2019,
 'orgao': '17',
 'codigo': '422863',
 'url': 'http://prefeituradejatai.sigepnet.com.br/transparencia/despesa_paga_detalhes.php?codigo=422863',
 'content': <memory at 0x72dec9259840>,
 'modified': datetime.datetime(2024, 9, 15, 6, 9, 1, 844256, tzinfo=<UTC>),
 'parent_id': 1761}

In [14]:
content = item['csv'].content
content = content.tobytes().decode('utf-8-sig')
file = StringIO(content)
csv_data = csv.reader(file, delimiter=";")
data = [tuple(row) for row in csv_data]

In [15]:
content

'"Código";"422863";"";"";""\n"Data";"13/12/2019";"";"";""\n"Banco";"Caixa Economica Federal";"";"";""\n"Agência";"2510";"";"";""\n"Conta";"635-9";"";"";""\n"Tipo do Documento";"Aviso Debito";"";"";""\n"Número do Documento";"63965442       ";"";"";""\n"Unidade Financeira";"30 - Fundo Esp Dos Procuradores Juridicos Poder Executivo";"";"";""\n"Fonte de Recursos";"100 - Recursos Não Vinculados De Impostos";"";"";""\n"";"";"";"";""\n"Número";"Série";"Data de Emissão";"Tipo";"Valor"\n"9767190097";"01      ";"31/12/1969";"Recibo";"6.153,84"\n<script type=\'text/javascript\'>window.close();</script>\n'

In [16]:
data

[('Código', '422863', '', '', ''),
 ('Data', '13/12/2019', '', '', ''),
 ('Banco', 'Caixa Economica Federal', '', '', ''),
 ('Agência', '2510', '', '', ''),
 ('Conta', '635-9', '', '', ''),
 ('Tipo do Documento', 'Aviso Debito', '', '', ''),
 ('Número do Documento', '63965442       ', '', '', ''),
 ('Unidade Financeira',
  '30 - Fundo Esp Dos Procuradores Juridicos Poder Executivo',
  '',
  '',
  ''),
 ('Fonte de Recursos',
  '100 - Recursos Não Vinculados De Impostos',
  '',
  '',
  ''),
 ('', '', '', '', ''),
 ('Número', 'Série', 'Data de Emissão', 'Tipo', 'Valor'),
 ('9767190097', '01      ', '31/12/1969', 'Recibo', '6.153,84'),
 ("<script type='text/javascript'>window.close()", '</script>')]

In [17]:
def clean_text(text):
    while '  ' in text:
        text = text.replace('  ', ' ')
    while ' \n' in text:
        text = text.replace(' \n', '\n')
    while '\n ' in text:
        text = text.replace('\n ', '\n')
    while '\r\n' in text:
        text = text.replace('\r\n', '\n')
    while '\n\n' in text:
        text = text.replace('\n\n', '\n')
    while '<td> ' in text:
        text = text.replace('<td> ', '<td>')
    while ' :</td>' in text:
        text = text.replace(' :</td>', ':</td>')
    while '\n</td>' in text:
        text = text.replace('\n</td>', '</td>')
    while ' </td>' in text:
        text = text.replace(' </td>', '</td>')
    while '<td>R$ ' in text:
        text = text.replace('<td>R$ ', '<td>')
    return text


In [18]:
content = item['html'].content
content = content.tobytes().decode('utf-8-sig')
content = clean_text(clean_text(clean_text(content)))

In [19]:
tables = bs(content).findAll('table')

In [20]:
tables

[<table class="table">
 <tbody>
 <tr>
 <td class="titulo">Código:</td>
 <td>422863</td>
 </tr>
 <tr>
 <td class="titulo">Data:</td>
 <td>13/12/2019</td>
 </tr>
 <tr>
 <td class="titulo">Banco:</td>
 <td>CAIXA ECONOMICA FEDERAL</td>
 </tr>
 <tr>
 <td class="titulo">Agência:</td>
 <td>2510</td>
 </tr>
 <tr>
 <td class="titulo">Conta Bancária:</td>
 <td>635-9</td>
 </tr>
 <tr>
 <td class="titulo">Tipo do Documento:</td>
 <td>AVISO DEBITO</td>
 </tr>
 <tr>
 <td class="titulo">Nº do Documento:</td>
 <td>63965442</td>
 </tr>
 <tr>
 <td class="titulo">Número da Licitação:</td>
 <td>0</td>
 </tr>
 <tr>
 <td class="titulo">Código Empenho:</td>
 <td>285874</td>
 </tr>
 <tr>
 <td class="titulo">Elemento:</td>
 <td>339036 - OUTROS SERVICOS DE TERCEIROS - PESSOA FISICA</td>
 </tr>
 <tr>
 <td class="titulo">Sub Elemento:</td>
 <td>06 -</td>
 </tr>
 <tr>
 <td class="titulo">Unidade Financeira:</td>
 <td>30 - FUNDO ESP DOS PROCURADORES JURIDICOS PODER EXECUTIVO</td>
 </tr>
 <tr>
 <td class="titulo">Fo

In [21]:
values = {}
for row in tables[0].findAll('tr'):
    cols = row.findAll('td')
    values[cols[0].text] = cols[1].text

In [22]:
values

{'Código:': '422863',
 'Data:': '13/12/2019',
 'Banco:': 'CAIXA ECONOMICA FEDERAL',
 'Agência:': '2510',
 'Conta Bancária:': '635-9',
 'Tipo do Documento:': 'AVISO DEBITO',
 'Nº do Documento:': '63965442',
 'Número da Licitação:': '0',
 'Código Empenho:': '285874',
 'Elemento:': '339036 - OUTROS SERVICOS DE TERCEIROS - PESSOA FISICA',
 'Sub Elemento:': '06 -',
 'Unidade Financeira:': '30 - FUNDO ESP DOS PROCURADORES JURIDICOS PODER EXECUTIVO',
 'Fonte de Recursos:': '100 - RECURSOS NÃO VINCULADOS DE IMPOSTOS',
 'Historico:': 'PAGAMENTO DA DESPESA PROVENIENTE DE EMPENHO EMITIDO PARA CUSTEAR DESPESAS A TÍTULO DE PARTICIPAÇÃO NOS HONORÁRIOS ADVOCATÍCIOS SUCUMBENCIAIS, APURADOS NO MÊS DE DEZEMBRO DE 2019.'}

In [23]:
rows = tables[1].findAll('tr')
col1 = rows[0].findAll('td')
col2 = rows[1].findAll('td')
for key, value in zip(col1, col2):
    values[key.text] = value.text

In [24]:
values

{'Código:': '422863',
 'Data:': '13/12/2019',
 'Banco:': 'CAIXA ECONOMICA FEDERAL',
 'Agência:': '2510',
 'Conta Bancária:': '635-9',
 'Tipo do Documento:': 'AVISO DEBITO',
 'Nº do Documento:': '63965442',
 'Número da Licitação:': '0',
 'Código Empenho:': '285874',
 'Elemento:': '339036 - OUTROS SERVICOS DE TERCEIROS - PESSOA FISICA',
 'Sub Elemento:': '06 -',
 'Unidade Financeira:': '30 - FUNDO ESP DOS PROCURADORES JURIDICOS PODER EXECUTIVO',
 'Fonte de Recursos:': '100 - RECURSOS NÃO VINCULADOS DE IMPOSTOS',
 'Historico:': 'PAGAMENTO DA DESPESA PROVENIENTE DE EMPENHO EMITIDO PARA CUSTEAR DESPESAS A TÍTULO DE PARTICIPAÇÃO NOS HONORÁRIOS ADVOCATÍCIOS SUCUMBENCIAIS, APURADOS NO MÊS DE DEZEMBRO DE 2019.',
 'Número': '9767190097',
 'Série': '01',
 'Data de Emissão': '13/12/2019',
 'Tipo': 'Recibo',
 'Valor': '6.153,84'}

In [25]:
sr = item['csv']
ds = models.Despesa.objects.filter(
    loa_id=sr.ano,
    orgao__codigo=sr.orgao,
    unidade__codigo=values['Unidade Financeira:'][:2],
    natureza__codigo='3.3.90.36.00'
).first()

In [26]:
ds.valor_materia

Decimal('176182.45')

In [27]:
values['Unidade Financeira:'][:2]

'30'

In [28]:
list(models.Despesa.objects.filter(loa_id=2024
    ).values_list('unidade__codigo',flat=True).order_by('unidade__codigo').distinct())

['01',
 '02',
 '03',
 '06',
 '08',
 '10',
 '11',
 '15',
 '16',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '25',
 '27',
 '28',
 '29',
 '30',
 '32',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '99']

In [29]:
models.ScrapRecord.objects.filter(metadata__url_dict__name__icontains='despesa').values(
    'codigo', 
).order_by('codigo',).annotate(count=Count('codigo')).filter(count__gt=2)

<QuerySet [{'codigo': '', 'count': 975}, {'codigo': 'TOTAL:', 'count': 4}]>

In [30]:
for d in models.Despesa.objects.filter(orgao_id=58,unidade_id=120, natureza_id=8381):
    print(d)

01.031.0139.2.001:3.1.90.92.00 - MANUT.ATIV.DA CAMARA MUNICIPAL
01.271.2839.9.061:3.1.90.92.00 - ENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA


In [31]:
from django.utils import timezone

In [32]:
start = timezone.localtime()

In [33]:
(timezone.localtime() - start).seconds

0

In [34]:
o = models.Orgao.objects.get(codigo='01', loa__ano=2024)

In [37]:
import datetime

In [38]:
datetime.datetime.strptime(values['Data:'], "%d/%m/%Y").date()

datetime.date(2019, 12, 13)

In [120]:
models.DespesaPaga.objects.filter(
    orgao__codigo='03', 
    orgao__loa__ano=2024,
        data__month=9
).aggregate(Sum('valor'))

{'valor__sum': Decimal('18593047.02')}

In [121]:
models.DespesaPaga.objects.count()

28971

In [122]:
for dp in models.DespesaPaga.objects.filter(
        orgao__codigo='03', 
        orgao__loa__ano=2024,
        data__month=9
    ).order_by('-data', '-codigo'):
    print(dp.data, dp.codigo, dp.valor)

2024-09-16 559089 12481.12
2024-09-16 559088 27470.14
2024-09-16 559087 294.50
2024-09-16 559086 58605.50
2024-09-16 559085 2479.86
2024-09-16 559084 10773.00
2024-09-16 559083 294.08
2024-09-16 559068 81.41
2024-09-16 559065 1125.00
2024-09-16 559064 1125.00
2024-09-16 559057 1161.00
2024-09-16 559056 1152.00
2024-09-16 559055 1179.00
2024-09-16 559054 1593.00
2024-09-16 559053 2700.00
2024-09-16 559052 2646.00
2024-09-16 559051 1206.00
2024-09-16 559050 1188.00
2024-09-16 559049 1143.00
2024-09-16 559047 2655.00
2024-09-16 559046 2628.00
2024-09-16 559045 1170.00
2024-09-16 559044 1134.00
2024-09-16 559043 1179.00
2024-09-16 559042 1539.00
2024-09-16 559041 375000.00
2024-09-16 559040 13566.41
2024-09-16 559039 99.04
2024-09-16 559037 167.71
2024-09-16 559029 1125.00
2024-09-16 559028 150.00
2024-09-16 559027 150.00
2024-09-16 559026 75.00
2024-09-16 559024 22382.00
2024-09-16 559022 13040.39
2024-09-16 559020 108.48
2024-09-16 559019 30700.00
2024-09-16 559018 4875.00
2024-09-16 559

In [109]:
for sr in models.ScrapRecord.objects.filter(codigo__startswith='559').order_by('-id')[:50]:
    print(sr.codigo, sr.orgao)

559054 03
559055 03
559055 03
559056 03
559056 03
559057 03
559057 03
559058 03
559058 03
559023 03
559023 03
559048 03
559048 03
559061 03
559061 03
559011 08
559011 08
559012 06
559012 06
559003 06
559003 06
559008 06
559008 06
559001 06
559001 06
559006 06
559006 06
559005 06
559005 06
559000 06
559000 06
559007 06
559007 06
559004 06
559004 06
559010 06
559010 06
559009 06
559009 06
559002 03
559002 03
559013 17
559013 17
